# Medical Cost Personal Datasets

In [39]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as pl
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('insurance.csv')

In [2]:
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


__목표: charges(병원 보험비) 예측__

- age: age of primary beneficiary

- sex: insurance contractor gender, female, male

- bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

- children: Number of children covered by health insurance / Number of dependents

- smoker: Smoking

- region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

- charges: Individual medical costs billed by health insurance

## 결측값 확인

In [3]:
data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

=> 결측값 없음

## categorical features 인코딩

In [4]:
data.sex.value_counts()

male      676
female    662
Name: sex, dtype: int64

In [5]:
data.smoker.value_counts()

no     1064
yes     274
Name: smoker, dtype: int64

In [6]:
data.region.value_counts()

southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64

region은 여러 class가 있어 원래라면 onehot encoding을 해야하지만, region은 order가 없기 때문에(?) label encoding 사용 

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#sex
le.fit(data.sex.drop_duplicates()) 
data.sex = le.transform(data.sex)

# smoker or not
le.fit(data.smoker.drop_duplicates()) 
data.smoker = le.transform(data.smoker)

#region
le.fit(data.region.drop_duplicates()) 
data.region = le.transform(data.region)

In [8]:
data.sex.value_counts()

1    676
0    662
Name: sex, dtype: int64

In [9]:
data.region.value_counts()

2    364
1    325
3    325
0    324
Name: region, dtype: int64

## 다항회귀 - 전진/후진 선택법

In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [9]:
data = sm.add_constant(data, has_constant = "add")
data.head()

,const,age,sex,bmi,children,smoker,region,charges
0,1.0,19,0,27.900,0,1,3,16884.92400
1,1.0,18,1,33.770,1,0,2,1725.55230
2,1.0,28,1,33.000,3,0,2,4449.46200
3,1.0,33,1,22.705,0,0,1,21984.47061
4,1.0,32,1,28.880,0,0,1,3866.85520


In [10]:
x = data.drop(['charges'], axis = 1)
y = data.charges

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0)

In [12]:
full_model = sm.OLS(y_train, x_train)
fitted_full_model = full_model.fit()

fitted_full_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.732
Method:                 Least Squares   F-statistic:                     457.4
Date:                Fri, 28 May 2021   Prob (F-statistic):          4.46e-282
Time:                        14:12:12   Log-Likelihood:                -10177.
No. Observations:                1003   AIC:                         2.037e+04
Df Residuals:                     996   BIC:                         2.040e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.126e+04   1116.164    -10.089      0.000   -1.35e+04   -9070.836
age          250.5376     14.062     17.817      0.000     222.943     278.132
sex            3.6509    392.214      0.009      0.993    -766.009     773.311
bmi          322.8819     31.928     10.113      0.000     260.227     385.537
children     450.1474    162.504      2.770      0.006     131.258     769.037
smoker       2.36e+04    488.662     48.300      0.000    2.26e+04    2.46e+04
region      -341.2440    179.063     -1.906      0.057    -692.629      10.141
==============================================================================
Omnibus:                      246.113   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              605.913
Skew:                           1.294   Prob(JB):                    2.68e-132
Kurtosis:                       5.793   Cond. No.                         294.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 변수선택법

$ AIC = -2*Log Likelihood + 2*p $

-2*Log(Likelihood) : 모형의 적합도
    
p : 모형의 추정된 파라미터의 개수

In [13]:
def processSubset(X, y, feature_set):
    model = sm.OLS(y,X[list(feature_set)])  
    regr = model.fit()   
    AIC = regr.aic
    return {"model" : regr, "AIC" : AIC}

In [14]:
import time
import itertools 

def getBest(X, y, k):
    tic = time.time()      
    results = []           
    for combo in itertools.combinations(X.columns.difference(['const'],k)) :    
        combo = (list(combo)+['const'])    
        results.append(processSubset(X,y,feature_set = combo)) 
        
    models = pd.DataFrame(results) 
    best_model = models.loc[models['AIC'].argmin()] 
    toc = time.time()       
    print("Processed", models.shape[0], "models on", k, "predictors in",(toc - tic),"seconds.")
    return best_model

### 전진선택법

전진선택법 : 절편만 있는 상수모형으로부터 시작해 중요하다고 생각되는 설명변수부터 차례로 모형에 추가

In [16]:
def forward(X, y, predictors):    
    remaining_predictors = [p for p in X.columns.difference(['const']) if p not in predictors]
    tic = time.time()   
    results = []
    for p in remaining_predictors :
        results.append(processSubset(X=X, y=y, feature_set=predictors+[p]+['const']))
    models = pd.DataFrame(results)    
    
    best_model = models.loc[models['AIC'].argmin()]   
    toc = time.time()     
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic))
    print("Selected predictors:",best_model["model"].model.exog_names,"AIC: ",best_model[0])
    return best_model    

In [17]:
def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=["AIC", "model"])
    tic = time.time()
    predictors = []   
    
    for i in range(1, len(X.columns.difference(['const'])) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors)
        if i > 1 :
            if Forward_result["AIC"] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapsed time:",(toc-tic), "seconds.")
    
    return (Fmodels['model'][len(Fmodels['model'])])

In [18]:
Forward_best_model = forward_model(X=x_train, y=y_train)

Processed  6 models on 1 predictors in 0.017948389053344727
Selected predictors: ['smoker', 'const'] AIC:  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000023DC88B5208>
Processed  5 models on 2 predictors in 0.00797724723815918
Selected predictors: ['smoker', 'age', 'const'] AIC:  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000023DC88C0048>
Processed  4 models on 3 predictors in 0.005984306335449219
Selected predictors: ['smoker', 'age', 'bmi', 'const'] AIC:  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000023DC88A0848>
Processed  3 models on 4 predictors in 0.005983829498291016
Selected predictors: ['smoker', 'age', 'bmi', 'children', 'const'] AIC:  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000023DC88CA648>
Processed  2 models on 5 predictors in 0.005987405776977539
Selected predictors: ['smoker', 'age', 'bmi', 'children', 'region', 'const'] AIC:  <statsmodels.re

In [19]:
Forward_best_model.aic

20365.16837203425

In [20]:
Forward_best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.732
Method:                 Least Squares   F-statistic:                     549.4
Date:                Fri, 28 May 2021   Prob (F-statistic):          1.81e-283
Time:                        14:12:25   Log-Likelihood:                -10177.
No. Observations:                1003   AIC:                         2.037e+04
Df Residuals:                     997   BIC:                         2.039e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
smoker       2.36e+04    487.264     48.439      0.000    2.26e+04    2.46e+04
age          250.5359     14.054     17.827      0.000     222.957     278.115
bmi          322.8922     31.893     10.124      0.000     260.306     385.478
children     450.1735    162.399      2.772      0.006     131.491     768.856
region      -341.2622    178.963     -1.907      0.057    -692.450       9.925
const      -1.126e+04   1104.179    -10.197      0.000   -1.34e+04   -9092.874
==============================================================================
Omnibus:                      246.104   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              605.856
Skew:                           1.294   Prob(JB):                    2.75e-132
Kurtosis:                       5.793   Cond. No.                         291.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 후진제거법

후진제거법 : 독립변수 후보 모두를 포함한 모형에서 출발해 가장 적은 영향을 주는 변수부터 하나씩 제거하며 더 이상 제거할 변수가 없을 때의 모형을 선택

In [21]:
def backward(X,y,predictors):
    tic = time.time()
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors) - 1):
        results.append(processSubset(X=X,y=y,feature_set=list(combo)+['const']))
    models = pd.DataFrame(results)
    best_model = models.loc[models['AIC'].argmin()]
    toc = time.time()
    
    print("Processed ",models.shape[0], "models on", len(predictors) - 1, "predictors in", (toc-tic))
    print("Selected predictors:",best_model['model'].model.exog_names,' AIC:',best_model[0])
    return best_model

In [22]:
def backward_model(X,y) :
    Bmodels = pd.DataFrame(columns=["AIC","model"], index = range(1,len(X.columns)))
    tic = time.time()
    predictors = X.columns.difference(['const'])
    Bmodel_before = processSubset(X,y,predictors)['AIC']
    while (len(predictors) > 1):
        Backward_result = backward(X=x_train, y=y_train, predictors=predictors)
        if Backward_result['AIC'] > Bmodel_before :
            break
        Bmodels.loc[len(predictors) -1] = Backward_result
        predictors = Bmodels.loc[len(predictors) - 1]['model'].model.exog_names
        Bmodel_before = Backward_result["AIC"]
        predictors = [k for k in predictors if k != 'const']
    
    toc = time.time()
    print("Total elapsed time:",(toc-tic),"seconds.")
    return (Bmodels["model"].dropna().iloc[0])

In [23]:
Backward_best_model = backward_model(X=x_train, y=y_train)

Processed  6 models on 5 predictors in 0.013963937759399414
Selected predictors: ['age', 'bmi', 'children', 'region', 'smoker', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000023DC931FD48>
Processed  5 models on 4 predictors in 0.01396322250366211
Selected predictors: ['age', 'bmi', 'children', 'smoker', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000023DC932DF88>
Total elapsed time: 0.03789687156677246 seconds.


In [24]:
Backward_best_model.aic

20365.16837203425

In [25]:
Backward_best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.732
Method:                 Least Squares   F-statistic:                     549.4
Date:                Fri, 28 May 2021   Prob (F-statistic):          1.81e-283
Time:                        14:12:30   Log-Likelihood:                -10177.
No. Observations:                1003   AIC:                         2.037e+04
Df Residuals:                     997   BIC:                         2.039e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age          250.5359     14.054     17.827      0.000     222.957     278.115
bmi          322.8922     31.893     10.124      0.000     260.306     385.478
children     450.1735    162.399      2.772      0.006     131.491     768.856
region      -341.2622    178.963     -1.907      0.057    -692.450       9.925
smoker       2.36e+04    487.264     48.439      0.000    2.26e+04    2.46e+04
const      -1.126e+04   1104.179    -10.197      0.000   -1.34e+04   -9092.874
==============================================================================
Omnibus:                      246.104   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              605.856
Skew:                           1.294   Prob(JB):                    2.75e-132
Kurtosis:                       5.793   Cond. No.                         291.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 성능평가

In [27]:
pred_y_full = fitted_full_model.predict(x_test)
pred_y_forward = Forward_best_model.predict(x_test[Forward_best_model.model.exog_names])
pred_y_backward = Backward_best_model.predict(x_test[Backward_best_model.model.exog_names])

In [28]:
import sklearn.metrics as metrics
perf_mat = pd.DataFrame(columns=["ALL", "FORWARD", "BACKWARD"],index =['MSE', 'RMSE','MAE'])

perf_mat.loc['MSE']['ALL'] = metrics.mean_squared_error(y_test,pred_y_full)
perf_mat.loc['MSE']['FORWARD'] = metrics.mean_squared_error(y_test,pred_y_forward)
perf_mat.loc['MSE']['BACKWARD'] = metrics.mean_squared_error(y_test,pred_y_backward)

perf_mat.loc['RMSE']['ALL'] = np.sqrt(metrics.mean_squared_error(y_test, pred_y_full))
perf_mat.loc['RMSE']['FORWARD'] = np.sqrt(metrics.mean_squared_error(y_test, pred_y_forward))
perf_mat.loc['RMSE']['BACKWARD'] = np.sqrt(metrics.mean_squared_error(y_test, pred_y_backward))

perf_mat.loc['MAE']['ALL'] = metrics.mean_absolute_error(y_test, pred_y_full)
perf_mat.loc['MAE']['FORWARD'] = metrics.mean_absolute_error(y_test, pred_y_forward)
perf_mat.loc['MAE']['BACKWARD'] = metrics.mean_absolute_error(y_test, pred_y_backward)

In [29]:
print(perf_mat)

              ALL      FORWARD     BACKWARD
MSE   3.20736e+07  3.20727e+07  3.20727e+07
RMSE      5663.36      5663.27      5663.27
MAE       3998.27      3998.04      3998.04


## 규제선형모델 적용: 릿지, 라쏘, 엘라스틱넷 회귀

In [42]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [43]:
x = data.drop(['charges'], axis = 1)
y = data["charges"]

In [45]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state = 0)

In [53]:
def get_linear_reg_eval(model_name, params=None, X_data_n=None, y_target_n=None, verbose=True):
    coeff_df = pd.DataFrame()
    if verbose : print('####### ', model_name , '#######')
    for param in params:
        if model_name =='Ridge': model = Ridge(alpha=param)
        elif model_name =='Lasso': model = Lasso(alpha=param)
        elif model_name =='ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_data_n, 
                                             y_target_n, scoring="neg_mean_squared_error", cv = 5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print('alpha {0}일 때 5 폴드 세트의 평균 RMSE: {1:.3f} '.format(param, avg_rmse))
        # cross_val_score는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출
        
        model.fit(x , y)
        # alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 컬럼으로 추가. 
        coeff = pd.Series(data=model.coef_ , index=x.columns )
        colname='alpha:'+str(param)
        coeff_df[colname] = coeff
    return coeff_df

### 릿지_L2규제 사용

In [61]:
ridge_alphas = [ 0.07, 0.1, 0.5, 1, 3, 5,10, 100]
coeff_lasso_df =get_linear_reg_eval('Ridge', params=ridge_alphas, X_data_n=x, y_target_n=y)

#######  Ridge #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 6068.390 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6068.394 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 6068.489 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 6068.720 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 6070.845 
alpha 5일 때 5 폴드 세트의 평균 RMSE: 6074.791 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 6091.809 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 7014.860 


In [72]:
sort_column = 'alpha:'+str(ridge_alphas[0])
coeff_lasso_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3,alpha:5,alpha:10,alpha:100
smoker,23812.732391,23809.433179,23765.531200,23710.881700,23494.783083,23282.601737,22768.601432,16299.298665
children,479.370825,479.371434,479.379375,479.388800,479.421020,479.444654,479.467713,473.910334
bmi,332.570793,332.571077,332.574850,332.579557,332.598270,332.616796,332.662259,333.248303
age,257.282802,257.280543,257.250480,257.213051,257.064967,256.919446,256.566419,252.056202
sex,-130.613255,-130.400272,-127.569484,-124.054358,-110.249124,-96.841995,-64.979753,252.512820
region,-353.632478,-353.629245,-353.586065,-353.531875,-353.312777,-353.090030,-352.517996,-339.950782


alpha값이 증가하면서 회귀 계수가 지속적으로 작아지는 추세를 보임. 회귀계수를 0으로 만들지 않음

### 라쏘_L1규제 사용

In [58]:
lasso_alphas = [ 0.07, 0.1, 0.5, 1, 3, 5,10]
coeff_lasso_df =get_linear_reg_eval('Lasso', params=lasso_alphas, X_data_n=x, y_target_n=y)

#######  Lasso #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 6068.376 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6068.374 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 6068.339 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 6068.296 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 6068.131 
alpha 5일 때 5 폴드 세트의 평균 RMSE: 6067.973 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 6067.614 


In [55]:
# 반환된 coeff_lasso_df를 첫번째 컬럼순으로 내림차순 정렬하여 회귀계수 DataFrame출력
sort_column = 'alpha:'+str(lasso_alphas[0])
coeff_lasso_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
smoker,23819.976521,23819.780405,23817.165521,23813.897000,23800.822579
children,479.319496,479.298129,479.013239,478.656095,477.231652
bmi,332.565534,332.563578,332.537499,332.504040,332.373655
age,257.288068,257.288041,257.287674,257.288631,257.286781
sex,-130.797852,-130.664072,-128.880339,-126.636592,-117.718069
region,-353.578746,-353.552499,-353.202540,-352.764393,-351.014609


피처 선택의 효과는 확인할 수 없음

### 엘라스틱 넷_L1+L2 규제

In [64]:
elastic_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_elastic_df =get_linear_reg_eval('ElasticNet', params=elastic_alphas,
                                      X_data_n=x, y_target_n=y)

#######  ElasticNet #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 6169.486 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6253.588 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 7627.566 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 8701.863 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 10153.916 


In [65]:
# 반환된 coeff_elastic_df를 첫번째 컬럼순으로 내림차순 정렬하여 회귀계수 DataFrame출력
sort_column = 'alpha:'+str(elastic_alphas[0])
coeff_elastic_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
smoker,21084.112769,20095.331567,12370.988170,8359.962488,3641.135569
children,479.134934,478.635500,460.949588,430.587605,328.718356
bmi,332.814278,332.905854,333.431965,332.966131,328.074827
age,255.404150,254.718071,249.247868,246.316732,242.677113
sex,32.921124,85.672744,354.229727,368.856369,239.126767
region,-350.248302,-348.636079,-323.859260,-293.426259,-208.159413


### 선형 회귀 모델을 위한 데이터 변환

In [66]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures

# method는 표준 정규 분포 변환(Standard), 최대값/최소값 정규화(MinMax), 로그변환(Log) 결정
# p_degree는 다향식 특성을 추가할 때 적용. p_degree는 2이상 부여하지 않음. 
def get_scaled_data(method='None', p_degree=None, input_data=None):
    if method == 'Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
    elif method == 'MinMax':
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif method == 'Log':
        scaled_data = np.log1p(input_data)
    else:
        scaled_data = input_data

    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, 
                                         include_bias=False).fit_transform(scaled_data)
    
    return scaled_data

In [68]:
# Ridge의 alpha값을 다르게 적용하고 다양한 데이터 변환방법에 따른 RMSE 추출. 
alphas = [0.1, 1, 10, 100]
#변환 방법은 모두 6개, 원본 그대로, 표준정규분포, 표준정규분포+다항식 특성
# 최대/최소 정규화, 최대/최소 정규화+다항식 특성, 로그변환 
scale_methods=[(None, None), ('Standard', None), ('Standard', 2), 
               ('MinMax', None), ('MinMax', 2), ('Log', None)]
for scale_method in scale_methods:
    X_data_scaled = get_scaled_data(method=scale_method[0], p_degree=scale_method[1], 
                                    input_data=x)
    print('\n## 변환 유형:{0}, Polynomial Degree:{1}'.format(scale_method[0], scale_method[1]))
    get_linear_reg_eval('Ridge', params=alphas, X_data_n=X_data_scaled, 
                        y_target_n=y, verbose=False)


## 변환 유형:None, Polynomial Degree:None
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6068.394 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 6068.720 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 6091.809 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 7014.860 

## 변환 유형:Standard, Polynomial Degree:None
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6068.381 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 6068.378 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 6069.040 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 6133.446 

## 변환 유형:Standard, Polynomial Degree:2
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 4840.885 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 4840.729 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 4839.584 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 4862.015 

## 변환 유형:MinMax, Polynomial Degree:None
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6068.365 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 6068.868 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 6119.802 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 7420.472 

## 변환 유형:MinMax, Polynomial Degree:2
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 4837.983 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 4866.799 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 5361.101 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 6171

## 부스팅 알고리즘 (회귀)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error

In [12]:
x = data.drop(['charges'], axis = 1)
y = data.charges

x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = 0)

In [70]:
x_train.shape

(1003, 6)

In [75]:
x_train.describe()

,age,sex,bmi,children,smoker,region
count,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000
mean,39.235294,0.492522,30.719766,1.076770,0.202393,1.568295
std,14.060124,0.500193,6.233512,1.207548,0.401984,1.105273
min,18.000000,0.000000,15.960000,0.000000,0.000000,0.000000
25%,26.500000,0.000000,26.302500,0.000000,0.000000,1.000000
50%,39.000000,0.000000,30.300000,1.000000,0.000000,2.000000
75%,51.000000,1.000000,34.800000,2.000000,0.000000,3.000000
max,64.000000,1.000000,53.130000,5.000000,1.000000,3.000000


In [76]:
x_test.describe()

,age,sex,bmi,children,smoker,region
count,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000
mean,39.122388,0.543284,30.494627,1.149254,0.211940,1.358209
std,14.040146,0.498868,5.679325,1.199473,0.409294,1.090260
min,18.000000,0.000000,17.195000,0.000000,0.000000,0.000000
25%,27.000000,0.000000,26.267500,0.000000,0.000000,0.000000
50%,39.000000,1.000000,30.590000,1.000000,0.000000,1.000000
75%,51.000000,1.000000,34.307500,2.000000,0.000000,2.000000
max,64.000000,1.000000,46.530000,5.000000,1.000000,3.000000


In [23]:
def get_model_cv_prediction_rmse(model, X_data, y_target):
    from sklearn.model_selection import cross_val_score
    neg_mse_scores = cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 5)
    rmse_scores  = np.sqrt(-1 * neg_mse_scores)
    avg_rmse = np.mean(rmse_scores)
    print('##### ',model.__class__.__name__ , ' #####')
    print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))

In [24]:
def get_model_cv_prediction_r2(model, X_data, y_target):
    from sklearn.model_selection import cross_val_score
    r2_scores = cross_val_score(model, X_data, y_target, scoring="r2", cv = 5)
    avg_rmse = np.mean(r2_scores)
    print('##### ',model.__class__.__name__ , ' #####')
    print(' 5 교차 검증의 평균 R2 : {0:.3f} '.format(avg_rmse))

### AdaBoost

In [49]:
from sklearn.ensemble import AdaBoostRegressor 
import time

start_time = time.time()

ada_reg = AdaBoostRegressor(random_state = 0)
ada_reg.fit(x_train, y_train)

y_train_pred = ada_reg.predict(x_train)
y_test_pred = ada_reg.predict(x_test)

print('train R2:', round(r2_score(y_train, y_train_pred),3))
print('test R2:', round(r2_score(y_test, y_test_pred),3))

print('소요시간:', round(time.time()-start_time,5))

train R2: 0.837
test R2: 0.877
소요시간: 0.05469


In [47]:
get_model_cv_prediction_r2(ada_reg, x_test, y_test)
get_model_cv_prediction_rmse(ada_reg, x_test, y_test)

#####  AdaBoostRegressor  #####
 5 교차 검증의 평균 R2 : 0.795 
#####  AdaBoostRegressor  #####
 5 교차 검증의 평균 RMSE : 5182.996 


### GBM 

In [50]:
from sklearn.ensemble import GradientBoostingRegressor

start_time = time.time()

gbm_reg = GradientBoostingRegressor(random_state = 0)
gbm_reg.fit(x_train, y_train)

y_train_pred = gbm_reg.predict(x_train)
y_test_pred = gbm_reg.predict(x_test)

print('train R2:', round(r2_score(y_train, y_train_pred),3))
print('test R2:', round(r2_score(y_test, y_test_pred),3))
print('소요시간:', round(time.time()-start_time,5))

train R2: 0.899
test R2: 0.898
소요시간: 0.18943


In [37]:
get_model_cv_prediction_r2(gbm_reg, x_test, y_test)
get_model_cv_prediction_rmse(gbm_reg, x_test, y_test)

#####  GradientBoostingRegressor  #####
 5 교차 검증의 평균 R2 : 0.857 
#####  GradientBoostingRegressor  #####
 5 교차 검증의 평균 RMSE : 4342.541 


### XGBoost

In [55]:
from xgboost import XGBRegressor 

start_time = time.time()

xgb_reg = XGBRegressor(objective='reg:squarederror')
xgb_reg.fit(x_train, y_train)

y_train_pred = xgb_reg.predict(x_train)
y_test_pred = xgb_reg.predict(x_test)

print('train R2:', round(r2_score(y_train, y_train_pred),3))
print('test R2:', round(r2_score(y_test, y_test_pred),3))
print('소요시간:', round(time.time()-start_time,5))

train R2: 0.891
test R2: 0.9
소요시간: 0.0981


In [42]:
get_model_cv_prediction_r2(xgb_reg, x_test, y_test)
get_model_cv_prediction_rmse(xgb_reg, x_test, y_test)

#####  XGBRegressor  #####
 5 교차 검증의 평균 R2 : 0.858 
#####  XGBRegressor  #####
 5 교차 검증의 평균 RMSE : 4355.758 


### LGBM

In [56]:
from lightgbm import LGBMRegressor

start_time = time.time()

lgb_reg = LGBMRegressor()
lgb_reg.fit(x_train, y_train)

y_train_pred = lgb_reg.predict(x_train)
y_test_pred = lgb_reg.predict(x_test)

print('train R2:', round(r2_score(y_train, y_train_pred),3))
print('test R2:', round(r2_score(y_test, y_test_pred),3))
print('소요시간:', round(time.time()-start_time,5))

train R2: 0.938
test R2: 0.88
소요시간: 0.17802


In [44]:
get_model_cv_prediction_r2(lgb_reg, x_test, y_test)
get_model_cv_prediction_rmse(lgb_reg, x_test, y_test)

#####  LGBMRegressor  #####
 5 교차 검증의 평균 R2 : 0.875 
#####  LGBMRegressor  #####
 5 교차 검증의 평균 RMSE : 4112.329 


### 알고리즘 비교

- **R2 기준 train 성능:** LGBM(0.938) > GBM(0.899) > XGBoost(0.891) > AdaBoost(0.837)
- **R2 기준 test 성능 (5-fold):** LGBM(0.875)> XGBoost(0.858) > GBM(0.857) > AdaBoost(0.795)
- **RMSE 기준 test 성능(5-fold):** LGBM(4112.329) < GBM(4342.541) < XGBoost(4355.758) < AdaBoost(5182.996)
- **소요시간:** AdaBoost(0.05469) < XGBoost(0.0981) < LGBM(0.17802) < GBM(0.18943)

LGBM 선택!

**하이퍼 파라미터 튜닝 후 알고리즘 성능을 비교하지 않은 이유:** 앙상블 계열 알고리즘은 기본적으로 과적합이나 노이즈에 뛰어난 알고리즘이기 때문에, 하이퍼 파라미터 튜닝으로 성능 수치 개선이 급격히 되는 경우는 많지 않음. 성능 차이가 별로 없으므로 알고리즘을 미리 선택하는 것이 더 효율적일 것으로 생각

### LGBM 하이퍼파라미터 튜닝

In [60]:
lgb_reg.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [68]:
from sklearn.model_selection import GridSearchCV

params = {
    'num_leaves': [21, 31, 50],
    'learning_rate': [0.1, 0.03, 0.003],
    'max_depth': [-1, 3, 5],
    'n_estimators': [50, 100, 300],
}

grid = GridSearchCV(LGBMRegressor(random_state=0), params, scoring='r2', cv=5)
grid.fit(x_train, y_train)

print(grid.best_estimator_)
print(round(r2_score(y_test, grid.predict(x_test)),3))

LGBMRegressor(max_depth=3, n_estimators=50, num_leaves=21, random_state=0)
0.906


- 튜닝 전 기본 하이퍼파라미터 R2값: 0.875
- 튜닝 후 R2 값: 0.906

## 배깅 알고리즘 (회귀)

In [45]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn import metrics

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 20)

**RandomForestRegressor**

In [37]:
rf = RandomForestRegressor()
rf.fit(X_train, Y_train)
rf_predict = rf.predict(X_test)
score_rf = r2_score(Y_test,rf_predict)

print("r_square score --> ",score_rf)
print('Mean Squared Error -->', metrics.mean_squared_error(Y_test, rf_predict))

r_square score -->  0.8816434483599419
Mean Squared Error --> 17797637.57277287


**하이퍼파라미터 최적화**

In [38]:
n_estimators = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
min_samples_split = [2, 5, 10]

rf_param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

In [39]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rf_param_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, Y_train)
rf_predict = rf_random.predict(X_test)
score_rf = r2_score(Y_test,rf_predict)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   13.6s finished


In [40]:
print("r_square score --> ",score_rf)
print('Mean Squared Error -->', metrics.mean_squared_error(Y_test, rf_predict))

r_square score -->  0.8940358472027168
Mean Squared Error --> 15934154.561441977


**StandardScaler**

In [41]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test) 

In [42]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rf_param_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, Y_train)
rf_predict = rf_random.predict(X_test)
score_rf = r2_score(Y_test,rf_predict)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   10.9s finished


In [43]:
print("r_square score --> ",score_rf)
print('Mean Squared Error -->', metrics.mean_squared_error(Y_test, rf_predict))

r_square score -->  0.8940855468225654
Mean Squared Error --> 15926681.077217294
